In [ ]:
! pip install transformer==4.44.2 -q
! pip install datasets==3.1.0 -q
! pip install tokenizers==0.19.1 -q

ERROR: Could not find a version that satisfies the requirement transformer==4.44.2 (from versions: none)
ERROR: No matching distribution found for transformer==4.44.2


In [ ]:
from datasets import load_dataset
import datasets

In [ ]:
ml_path = "aalksii/ml-arxiv-papers"
wiki_path = "Salesforce/wikitext"

ml_ds = load_dataset(ml_path, split="train")
wiki_ds = load_dataset(wiki_path, split="train", name="wikitext-2-raw-v1")

In [ ]:
ml_ds, wiki_ds

(Dataset({
     features: ['title', 'abstract'],
     num_rows: 105832
 }),
 Dataset({
     features: ['text'],
     num_rows: 36718
 }))

In [ ]:
ml_ds = ml_ds.remove_columns(["title"])


In [ ]:
ml_ds=ml_ds.rename_column("abstract","text")


In [ ]:
con_ds = datasets.concatenate_datasets([ml_ds, wiki_ds])

In [ ]:
con_ds

Dataset({
    features: ['text'],
    num_rows: 142550
})

In [ ]:
# Q1

con_ds.num_rows

142550

In [ ]:
# Q2

def count_tokens(row):
    return {"tokens": len(row["text"].split(" "))}

In [ ]:
new_ds = con_ds.map(count_tokens, num_proc=12)

In [ ]:
total_tokens = 0
for tok in new_ds["tokens"]:
    total_tokens+=tok

print(total_tokens)

19797683


In [ ]:
total_tokens/len(new_ds)

138.88237811294283

In [ ]:
# Q3 How many rows have more than or equal to 150 words but less than or equal to 400 words (answer in thousands of number of rows)

def check_num_words(tokens):
    return tokens >= 150 and tokens <= 400

In [ ]:
filt_ds = new_ds.filter(check_num_words, input_columns="tokens")

In [ ]:
filt_ds

Dataset({
    features: ['text', 'tokens'],
    num_rows: 70519
})

In [ ]:
from tokenizers.normalizers import BertNormalizer
from tokenizers.pre_tokenizers import BertPreTokenizer

In [ ]:
# HF docs: This pre-tokenizer splits tokens on spaces, and also on punctuation. Each occurence of a punctuation character will be treated separately.


In [ ]:
# Q5
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


In [ ]:
tokenizer1 = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer2 = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [ ]:
tokenizer1.normalizer = BertNormalizer(lowercase=True)
tokenizer1.pre_tokenizer = BertPreTokenizer()

tokenizer2.normalizer = BertNormalizer(lowercase=True)
tokenizer2.pre_tokenizer = BertPreTokenizer()

In [ ]:
def get_training_corpus():
    for i in range(0, len(new_ds), 1000):
        yield new_ds[i : i + 1000]["text"]

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer5 = trainers.WordPieceTrainer(vocab_size=5000, special_tokens=special_tokens)
trainer10=trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens)


In [ ]:
tokenizer11 = tokenizer1.train_from_iterator(get_training_corpus(), trainer=trainer5) # 5K
tokenizer22 = tokenizer2.train_from_iterator(get_training_corpus(), trainer=trainer10) # 10K

In [ ]:
sent = "From misfiring superstars Rohit and Kohli, to New Zealand’s spin attack making the most of the home pitches, here’s how India were handed a shock series loss."

In [ ]:
tok5 = tokenizer1.encode(sent)

In [ ]:
tok10 = tokenizer2.encode(sent)

In [ ]:
print(tok5)
print(tok10)

Encoding(num_tokens=53, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=48, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [ ]:
from transformers import DistilBertConfig, AutoTokenizer, DistilBertForMaskedLM

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

config=DistilBertConfig()
model=DistilBertForMaskedLM(config)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Q8
config

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.44.2",
  "vocab_size": 30522
}

In [ ]:
# Q9

param2 = 0
for param in model.parameters():
    param2+=param.numel()
print(param2/1e6)

66.98553


In [ ]:
config.max_position_embeddings = 256
config

model_for_c1 = DistilBertForMaskedLM(config)
param1 = 0
for param in model_for_c1.parameters():
    param1+=param.numel()
print(param1/1e6)

66.788922


In [ ]:
config.max_position_embeddings = 1024
config

model_for_c2 = DistilBertForMaskedLM(config)
param3 = 0
for param in model_for_c2.parameters():
    param3+=param.numel()
print(param3/1e6)

67.378746


In [ ]:
# Q10
print(param1 < param2 < param3)
print(abs(param1 - param2) < abs(param2 - param3))

True
True


In [ ]:
def count_tokens(row):
    return {"tokens": tokeni}

Quiz 2 practice

In [1]:
! pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [16]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [20]:
imdb_ds = load_dataset("stanfordnlp/imdb", split="train[0:1000]")

In [22]:
imdb_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [4]:
def get_num_words(example):
    num_words = len(example["text"].split())
    return {"num_words": num_words}

ds = imdb_ds.map(get_num_words)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [5]:
ds

Dataset({
    features: ['text', 'label', 'num_words'],
    num_rows: 25000
})

In [14]:

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [19]:
ds3 = concatenate_datasets([imdb_ds, dataset])

ValueError: Dataset at position 0 has at least one split: ['train', 'test', 'unsupervised']
Please pick one to interleave with the other datasets, for example: dataset['train']

In [18]:
ds3

NameError: name 'ds3' is not defined